In [ ]:
# Start writing code here...from sklearn.cluster import KMeans
import pandas as pd
import numpy as np
from sklearn.manifold import MDS
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.cluster.hierarchy import dendrogram
from scipy.spatial.distance import cdist
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
import pandas as pd
import numpy as np
spotify_songs = pd.read_csv('https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2020/2020-01-21/spotify_songs.csv')
Q1 = np.percentile(spotify_songs.duration_ms, 25,
                   interpolation = 'midpoint')
Q3 = np.percentile(spotify_songs.duration_ms, 75,
                   interpolation = 'midpoint')
IQR = Q3 - Q1
mask = (spotify_songs.duration_ms < (Q3+1.5*IQR))
mask2 = spotify_songs.duration_ms > (Q1-1.5*IQR)
spotify_data_cleaned = spotify_songs.loc[mask]
spotify_data_cleaned = spotify_data_cleaned.loc[mask2]
spotify_data_cleaned.track_album_release_date.str[:4]
spotify_data_cleaned.track_album_release_date.str[5:7]
spotify_data_cleaned = spotify_data_cleaned.assign(
    track_album_release_month = spotify_data_cleaned.track_album_release_date.str[5:7],
    track_album_release_year = spotify_data_cleaned.track_album_release_date.str[:4],
)
mask1 = spotify_data_cleaned.track_popularity <= 80
mask2 = spotify_data_cleaned.track_popularity <=60
mask3 = spotify_data_cleaned.track_popularity <=40
mask4 = spotify_data_cleaned.track_popularity <=20

spotify_data_cleaned["popularity_quali"] = "trèsPopulaire"
spotify_data_cleaned.loc[mask1,"popularity_quali"] = "bienPopulaire"
spotify_data_cleaned.loc[mask2,"popularity_quali"] = "moyennementPopulaire"
spotify_data_cleaned.loc[mask3,"popularity_quali"] = "bofPopulaire"
spotify_data_cleaned.loc[mask4,"popularity_quali"] = "pasPopulaire"

spotify_data_cleaned.drop(columns="track_album_release_date",inplace=True)
mask = (spotify_data_cleaned.track_popularity > 0)
spotify_data_cleaned = spotify_data_cleaned.loc[mask]
#bon en vrai ca marche bien pour 100 mais pas pour tous mais flemme de faire une belle fonction mdr on verra si on a le temps
def spotify_data_short(nbTrackByGenre):
    spotify_data_short = pd.concat((spotify_data_cleaned[5000:5000+nbTrackByGenre],spotify_data_cleaned[10000:10000+nbTrackByGenre],spotify_data_cleaned[15000:15000+nbTrackByGenre],spotify_data_cleaned[20000:20000+nbTrackByGenre],spotify_data_cleaned[25000:25000+nbTrackByGenre]), axis=0)
    return spotify_data_short
    short = spotify_data_short(100)
short = spotify_data_short(100)
spotify_data_quanti = short[["track_popularity","danceability","energy","loudness","speechiness","acousticness","instrumentalness","liveness","valence","tempo","duration_ms"]]
data = spotify_data_cleaned[["track_popularity","danceability","energy","speechiness","acousticness","instrumentalness","liveness","valence"]]


In [ ]:
spotify_data_cleaned.playlist_genre

0        pop
1        pop
2        pop
3        pop
4        pop
        ... 
32826    edm
32827    edm
32828    edm
32830    edm
32832    edm
Name: playlist_genre, Length: 28953, dtype: object

In [ ]:
mask1 = spotify_data_cleaned.playlist_genre == "edm"
spotify_data_cleaned.loc[~mask1,"playlist_genre"] = "nonEdm"
spotify_data_cleaned.loc[~mask1,"playlist_genre"]


0        nonEdm
1        nonEdm
2        nonEdm
3        nonEdm
4        nonEdm
          ...  
26785    nonEdm
26786    nonEdm
26787    nonEdm
26788    nonEdm
26789    nonEdm
Name: playlist_genre, Length: 23959, dtype: object

In [ ]:

def diviser_par_max():
    columns = ["danceability","energy","speechiness","acousticness","instrumentalness","liveness","valence","track_popularity"]
    for column in columns:
        Q1 = np.percentile(data[column], 25,interpolation = 'midpoint')
        Q3 = np.percentile(data[column], 75,interpolation = 'midpoint')
        IQR = Q3 - Q1
        mask = (data[column] < (Q3+1.5*IQR))
        mask2 = data[column] > (Q1-1.5*IQR)
        spotify_data_divided = data.loc[mask]
        spotify_data_divided = data.loc[mask2]
        spotify_data_divided[column] = spotify_data_divided[column]/spotify_data_divided[column].max()
    return spotify_data_divided

spotify_data_divided = diviser_par_max()
spotify_data_divided

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


,track_popularity,danceability,energy,speechiness,acousticness,instrumentalness,liveness,valence
0,0.66,0.748,0.916,0.0583,0.102000,0.000000,0.0653,0.5180
1,0.67,0.726,0.815,0.0373,0.072400,0.004210,0.3570,0.6930
2,0.70,0.675,0.931,0.0742,0.079400,0.000023,0.1100,0.6130
3,0.60,0.718,0.930,0.1020,0.028700,0.000009,0.2040,0.2770
4,0.69,0.650,0.833,0.0359,0.080300,0.000000,0.0833,0.7250
...,...,...,...,...,...,...,...,...
32826,0.47,0.645,0.832,0.0294,0.001060,0.002640,0.1990,0.3750
32827,0.27,0.581,0.640,0.0365,0.026600,0.000000,0.5720,0.2880
32828,0.42,0.428,0.922,0.0936,0.076600,0.000000,0.0668,0.2100
32830,0.14,0.529,0.821,0.0481,0.108000,0.000001,0.1500,0.4360


In [ ]:
x = spotify_data_divided
y = spotify_data_cleaned.playlist_genre

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=0)


In [ ]:
from sklearn.model_selection import GridSearchCV
# De 1 à 500 voisins (exclu), échelle logarithmique
n_neighbors_list = np.unique(np.round(np.geomspace(1, 500, 100)).astype(int))
param_grid = {"n_neighbors": n_neighbors_list}
cls = KNeighborsClassifier()
search = GridSearchCV(cls, param_grid, scoring="accuracy", cv=10)
search.fit(x, y)
kopt = search.best_params_


In [ ]:
from sklearn.neighbors import KNeighborsClassifier#instanciation et définition du k
print(kopt)
knn = KNeighborsClassifier(n_neighbors = kopt["n_neighbors"])
#training
knn.fit(x_train,y_train)

{'n_neighbors': 142}


KNeighborsClassifier(n_neighbors=142)

In [ ]:
knn.score(x_test,y_test)

0.852051388313303

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=c2ad4461-7f35-42f5-b2d9-f686e8952f67' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>